<a href="https://colab.research.google.com/github/TaniaHurtado2024/Seminario_BigData/blob/main/1_PPvsSpark_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas vs Polars vs Spark vs Dask

Inspired in: https://www.youtube.com/watch?v=mi9f9zOaqM8

Original data: Kaggle

This jupyter notebook is designed to study and compare different tools to read and manipulate data; to be used in the data undertanding phase. The corresponding explanations will be given directly in class, therefore the material isn't autoexplained. Don´t forget ask me for the access to the data. And, please, give credits to the original author's idea and, if consider, also to me.

_Updated: June 20th, 2023_

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Playing with pandas

In [ ]:
import pandas as pd
flights_file1 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2018.parquet"
flights_file2 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2019.parquet"
#flights_file3 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2020.parquet"
flights_file4 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2021.parquet"
#flights_file5 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2022.parquet"
df1 = pd.read_parquet(flights_file1)
df2 = pd.read_parquet(flights_file2)
#df3 = pd.read_parquet(flights_file3)
df4 = pd.read_parquet(flights_file4)
#df5 = pd.read_parquet(flights_file5)

In [ ]:
df = pd.concat([df1, df2, df4])
#df = df1

In [ ]:
%%timeit

df_agg = df.groupby(['Airline','Year'])[["DepDelayMinutes", "ArrDelayMinutes"]].agg(
    ["mean", "sum", "max"]
)
df_agg = df_agg.reset_index()
df_agg.to_parquet("temp_pandas.parquet")

In [ ]:
!ls -GFlash temp_pandas.parquet

In [ ]:
pd.read_parquet('temp_pandas.parquet')

In [ ]:
pd.read_parquet('temp_pandas.parquet').info()

## Playing with Polars

In [1]:
import polars as pl

In [2]:
flights_file1 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2018.parquet"
flights_file2 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2019.parquet"
#flights_file3 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2020.parquet"
flights_file4 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2021.parquet"
#flights_file5 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2022.parquet"
df1 = pl.scan_parquet(flights_file1)
df2 = pl.scan_parquet(flights_file2)
#df3 = pl.scan_parquet(flights_file3)
df4 = pl.scan_parquet(flights_file4)
#df5 = pl.scan_parquet(flights_file5)

In [3]:
 %%timeit

df_polars = (
    pl.concat([df1, df2, df4])
    .groupby(['Airline', 'Year'])
    .agg([
        pl.col("DepDelayMinutes").mean().alias("avg_dep_delay"),
        pl.col("DepDelayMinutes").sum().alias("sum_dep_delay"),
        pl.col("DepDelayMinutes").max().alias("max_dep_delay"),
        pl.col("ArrDelayMinutes").mean().alias("avg_arr_delay"),
        pl.col("ArrDelayMinutes").sum().alias("sum_arr_delay"),
        pl.col("ArrDelayMinutes").max().alias("max_arr_delay"),
      ])
).collect()

df_polars.write_parquet('temp_polars.parquet')

<magic-timeit>:3: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.


6.86 s ± 979 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
!ls -GFlash temp_polars.parquet

8.0K -rw-r--r-- 1 root 6.5K Jul 17 01:36 temp_polars.parquet


## Playing with PySpark

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=d4b70109f34e85f7f1847203118dbbd381a9f30c8559cb6b5dafe86fb2f8f6fa
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, max, sum, concat

In [4]:
spark = SparkSession.builder.master("local[1]").appName("airline-example").getOrCreate()

In [5]:
flights_file1 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2018.parquet"
flights_file2 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2019.parquet"
#flights_file3 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2020.parquet"
flights_file4 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2021.parquet"
#flights_file5 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2022.parquet"

In [6]:
df_spark1 = spark.read.parquet(flights_file1)
df_spark2 = spark.read.parquet(flights_file2)
#df_spark3 = spark.read.parquet(flights_file3)
df_spark4 = spark.read.parquet(flights_file4)
#df_spark5 = spark.read.parquet(flights_file5)

In [7]:
df_spark = df_spark1.union(df_spark2)
#df_spark = df_spark.union(df_spark3)
df_spark = df_spark.union(df_spark4)
#df_spark = df_spark.union(df_spark5)

In [8]:
 %%timeit

df_spark_agg = df_spark.groupby("Airline", "Year").agg(
    avg("ArrDelayMinutes").alias('avg_arr_delay'),
    sum("ArrDelayMinutes").alias('sum_arr_delay'),
    max("ArrDelayMinutes").alias('max_arr_delay'),
    avg("DepDelayMinutes").alias('avg_dep_delay'),
    sum("DepDelayMinutes").alias('sum_dep_delay'),
    max("DepDelayMinutes").alias('max_dep_delay'),
)
df_spark_agg.write.mode('overwrite').parquet('temp_spark.parquet')

7.34 s ± 1.35 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
!ls -GFlash temp_pyspark.parquet

ls: cannot access 'temp_pyspark.parquet': No such file or directory


## Playing with dask

In [1]:
import pandas as pd
import dask.dataframe as dd
flights_file1 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2018.parquet"
flights_file2 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2019.parquet"
#flights_file3 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2020.parquet"
flights_file4 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2021.parquet"
#flights_file5 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2022.parquet"
df1 = dd.read_parquet(flights_file1)
df2 = dd.read_parquet(flights_file2)
#df3 = dd.read_parquet(flights_file3)
df4 = dd.read_parquet(flights_file4)
#df5 = dd.read_parquet(flights_file5)

In [2]:
df = dd.concat([df1,df2,df4])

In [ ]:
print(df.compute())

In [ ]:
df = df.compute()

In [ ]:
%%timeit

df_agg = df.groupby(['Airline','Year'])[["DepDelayMinutes", "ArrDelayMinutes"]].agg(
    ["mean", "sum", "max"]
)
df_agg = df_agg.reset_index()
df_agg.to_parquet("temp_dask.parquet")

In [ ]:
!ls -GFlash temp_dask.parquet

In [ ]:
pd.read_parquet('temp_dask.parquet').info()

In [ ]:
pd.read_parquet('temp_dask.parquet')

## Read Results

In [ ]:
import pandas as pd

In [ ]:
#agg_pandas = pd.read_parquet('temp_pandas.parquet')
agg_polars = pd.read_parquet('temp_polars.parquet')
agg_spark = pd.read_parquet('temp_spark.parquet')
agg_dask = pd.read_parquet('temp_dask.parquet')

In [ ]:
#agg_pandas.shape, agg_polars.shape, agg_spark.shape, agg_dask.shape

In [ ]:
#agg_pandas.sort_values(['Airline','Year']).head()

In [ ]:
agg_polars.sort_values(['Airline','Year']).head()

,Airline,Year,avg_dep_delay,sum_dep_delay,max_dep_delay,avg_arr_delay,sum_arr_delay,max_arr_delay
5,Air Wisconsin Airlines Corp,2018,16.753459,1606774.0,1296.0,17.881934,1708887.0,1292.0
97,Air Wisconsin Airlines Corp,2019,16.868511,1742281.0,1690.0,17.610384,1811545.0,1707.0
51,Air Wisconsin Airlines Corp,2020,8.583725,433315.0,1460.0,8.982529,452450.0,1439.0
49,Air Wisconsin Airlines Corp,2021,16.553045,1290194.0,1421.0,17.327440,1346602.0,1416.0
106,Air Wisconsin Airlines Corp,2022,13.124801,510581.0,1355.0,13.340409,517261.0,1353.0


In [ ]:
agg_spark.sort_values(['Airline','Year']).head()

,Airline,Year,avg_arr_delay,sum_arr_delay,max_arr_delay,avg_dep_delay,sum_dep_delay,max_dep_delay
0,Air Wisconsin Airlines Corp,2018,17.881934,1708887.0,1292.0,16.753459,1606774.0,1296.0
48,Air Wisconsin Airlines Corp,2019,17.610384,1811545.0,1707.0,16.868511,1742281.0,1690.0
56,Air Wisconsin Airlines Corp,2020,8.982529,452450.0,1439.0,8.583725,433315.0,1460.0
93,Air Wisconsin Airlines Corp,2021,17.327440,1346602.0,1416.0,16.553045,1290194.0,1421.0
119,Air Wisconsin Airlines Corp,2022,13.340409,517261.0,1353.0,13.124801,510581.0,1355.0


In [ ]:
agg_dask.sort_values(['Airline','Year']).head()

Airline  Year DepDelayMinutes                     \
                                                mean        sum     max   
0  Air Wisconsin Airlines Corp  2020        8.583725   433315.0  1460.0   
1  Air Wisconsin Airlines Corp  2022       13.124801   510581.0  1355.0   
2         Alaska Airlines Inc.  2020        5.818328   772930.0   823.0   
3         Alaska Airlines Inc.  2022       10.153994  1278134.0   915.0   
4                Allegiant Air  2020       12.825575  1080016.0  1648.0   

  ArrDelayMinutes                     
             mean        sum     max  
0        8.982529   452450.0  1439.0  
1       13.340409   517261.0  1353.0  
2        6.365082   843157.0   788.0  
3       11.026280  1382905.0   908.0  
4       13.331111  1115734.0  1645.0